# Implementación de Simplex

## Librerias

In [ ]:
import numpy as np
import scipy.optimize as op

## Simplex

Función donde se implementa el algoritmo Simplex para optimización de problemas de programación lineal en la forma estandar.

* El código fue adapatado de: https://github.com/mmolteratx/Simplex



In [ ]:
# ------------------------------------------------------------------
#                      Funciones Auxiliares
# ------------------------------------------------------------------

def printTableau(tableau, f):

    f = f[np.nonzero(f)]
    print("ind \t", end = "")
    for j in range(0, len(f)):
        print("x_" + str(j), end = "\t")
    for j in range(0, (len(tableau[0]) - len(f) - 2)):
        print("s_" + str(j), end = "\t")

    print()
    for j in range(0, len(tableau)):
        for i in range(0, len(tableau[0])):
            if(not np.isnan(tableau[j, i])):
                if(i == 0):
                    print(int(tableau[j, i]), end = "\t")
                else:
                    print(round(tableau[j, i], 2), end = "\t")
            else:
                print(end = "\t")
        print()

def getTableau(c, A, b):
  # Construct starting tableau
  c[0:len(c)] = -1 * c[0:len(c)]

  t2 = np.array([None, 0])
  numVar = len(c)
  numSlack = len(A)

  t2 = np.hstack(([None], c, [0]))

  basis = np.array([0] * numSlack)

  for i in range(0, len(basis)):
      basis[i] = numVar + i

  t1 = np.hstack((np.transpose([basis]), A, np.transpose([b])))

  tableau = np.vstack((t1, t2))

  tableau = np.array(tableau, dtype ='float')

  return tableau

# ------------------------------------------------------------------
#                      Funcion Simplex
# ------------------------------------------------------------------

def simplex(f, A, b, print_iter=True):
  for i in range(len(f)):
    f[i] = -1 * f[i]

  # Build Tableu
  tableau = getTableau(f, A, b)

  if(print_iter):
      print("Starting Tableau:")
      printTableau(tableau, f)

  # Assume initial basis is not optimal
  optimal = False

  # Keep track of iterations for display
  iter = 1

  while not optimal:

    if print_iter:
      print("--------------------------------------------------------------")
      print("ITERATION ", iter)
      printTableau(tableau, f)

    # Look for direction of decreased cost
    for cost in tableau[-1, 1:-1]:
      if cost < 0:
        optimal = False
        break
      optimal = True

    # If all directions result in decreased cost SBF is optimal
    if optimal:
      break

    # nth variable enters basis, account for tableau indexing
    n = tableau[-1, 1:-1].tolist().index(np.amin(tableau[-1, 1:-1])) + 1 # PUEDE QUE NO SEA 2

    # Minimum ratio test, rth variable leaves basis
    minimum = 99999
    r = -1

    for i in range(0, len(tableau)-1):
      if (tableau[i, n] > 0):
        val = tableau[i, -1]/tableau[i, n]
        if val<minimum:
          minimum = val
          r = i

    pivot = tableau[r, n]

    print("Pivot Column:", n)
    print("Pivot Row:", r)
    print("Pivot Element: ", pivot)

    # Perform row operations
    # Divide the pivot row with the pivot element
    tableau[r, 1:] = tableau[r, 1:] / pivot

    # Pivot other rows
    for i in range(0, len(tableau)):
      if i != r:
        mult = tableau[i, n] / tableau[r, n]
        tableau[i, 1:] = tableau[i, 1:] - mult * tableau[r, 1:]

    # New basic variable
    tableau[r, 0] = n - 1
    iter += 1

  if print_iter:
    print("--------------------------------------------------------------")
    print("Final Tableau reached in", iter, "iterations")
    printTableau(tableau, f)
  else:
      print("Solved")

  x = np.array([0] * len(f), dtype = float)
  # Save coefficients
  for key in range(0, (len(tableau))):
      if (tableau[key, 0] < len(f)):
          x[int(tableau[key, 0])] = tableau[key, -1]

  optimalValue = -1 * tableau[-1,-1]

  print("--------------------------------------------------------------")
  print("SOLUTION: ")
  print("Fobj Optimal Value: " + str(optimalValue))
  print("Solution: " + str(x))


##  Ejemplo

Ejemplo de como utilizar la función implementada en python.

Problema:
$$ \min f(x) = -3 x_1 -  x_2 - 3 x_3 $$

$$ \begin{array}{rl}
  2x_1 + x_2 + x_3 & \leq 2 \\
  x_1 + 2x_2 + 3x_3 & \leq 5 \\
  2x_1 + 2x_2 + x_3 & \leq 6 \\
  x_1,x_2, x_3 & \geq 0 \\
 \end{array}$$

 Forma Estándar:
 $$ \min f(x) = -3 x_1 -  x_2 - 3 x_3  $$

$$ \begin{array}{rl}
  2x_1 + x_2 + x_3 + y_1 & = 2 \\
  x_1 + 2x_2 + 3x_3 + y_2 & = 5 \\
  2x_1 + 2x_2 + x_3 + y_3 & = 6 \\
  x_1,x_2, x_3, y_1, y_2, y_3 & \geq 0 \\
 \end{array}$$

In [ ]:
# Probar función
f = np.array([-3, -1, -3, 0, 0, 0])
A = np.array([[2, 1, 1, 1, 0, 0], [1, 2, 3, 0, 1, 0], [2, 2, 1, 0, 0, 1]])
b = np.array([2, 5, 6])

simplex(f,A,b)

Starting Tableau:
ind 	x_0	x_1	x_2	s_0	s_1	s_2	
6	2.0	1.0	1.0	1.0	0.0	0.0	2.0	
7	1.0	2.0	3.0	0.0	1.0	0.0	5.0	
8	2.0	2.0	1.0	0.0	0.0	1.0	6.0	
	-3.0	-1.0	-3.0	0.0	0.0	0.0	0.0	
--------------------------------------------------------------
ITERATION  1
ind 	x_0	x_1	x_2	s_0	s_1	s_2	
6	2.0	1.0	1.0	1.0	0.0	0.0	2.0	
7	1.0	2.0	3.0	0.0	1.0	0.0	5.0	
8	2.0	2.0	1.0	0.0	0.0	1.0	6.0	
	-3.0	-1.0	-3.0	0.0	0.0	0.0	0.0	
Pivot Column: 1
Pivot Row: 0
Pivot Element:  2.0
--------------------------------------------------------------
ITERATION  2
ind 	x_0	x_1	x_2	s_0	s_1	s_2	
0	1.0	0.5	0.5	0.5	0.0	0.0	1.0	
7	0.0	1.5	2.5	-0.5	1.0	0.0	4.0	
8	0.0	1.0	0.0	-1.0	0.0	1.0	4.0	
	0.0	0.5	-1.5	1.5	0.0	0.0	3.0	
Pivot Column: 3
Pivot Row: 1
Pivot Element:  2.5
--------------------------------------------------------------
ITERATION  3
ind 	x_0	x_1	x_2	s_0	s_1	s_2	
0	1.0	0.2	0.0	0.6	-0.2	0.0	0.2	
2	0.0	0.6	1.0	-0.2	0.4	0.0	1.6	
8	0.0	1.0	0.0	-1.0	0.0	1.0	4.0	
	0.0	1.4	0.0	1.2	0.6	0.0	5.4	
--------------------------------

# Punto 1 - Análisis del Algoritmo

## Análisis Matemático

Desde el punto de vista matemático, cabe resaltar que el algoritmo implementado de ejemplo es incorrecto. Esto se concluye revisando los resultados de la última iteración y del retorno. El retorno, al ser la solución óptima con su respectivo costo, es correcto sobre las variables básicas originales. Sin embargo, no escribe el valor de las variables slack que son agregadas a la hora de llevar el problema a forma estándar.

En consecuencia, en una solución óptima en la cual las variables slack sean las que contienen el valor, el algoritmo no mostrará ese valor sino que las mostrará en cero. Sería necesario revisar el pivoteo en el tableau para conocer su valor correspondiente.

## Análisis Algorítmico

Desde el punto de vista algorítmico, el código implementado es correcto. Esto se concluye a partir de la observación de los resultados obtenidos en cada iteración del proceso mostrado en la consola. En este sentido, el proceso del simplex es adecuado puesto que sigue la siguiente estructura:

* Seleccionar $m$ variables básicas y pivotear sobre ellas
* Calcular el cociente de la columna de $\frac{\vec{b}}{\vec{a}_i}$, donde $\vec{a}_i$ es la columna (variable) con el coeficiente más negativo en la función objetivo.
* Sacar de la base la variable correspondiente a la posición de la insertada.
* Pivotear sobre la nueva base definida y repetir hasta que la función objetivo (última fila del tableau) tenga elementos positivos.
* Expresar la solución óptima en términos de las variables básicas originales.

En particular, las líneas de código

    # Look for direction of decreased cost
    for cost in tableau[-1, 1:-1]:
      if cost < 0:
        optimal = False
        break
      optimal = True

    # If all directions result in decreased cost SBF is optimal
    if optimal:
      break

Emplean la estructura de control ```break``` para detener la ejecución del programa una vez que se logran las condiciones de optimalidad y que no siga iterando.



## Criterio del Ingeniero

Si bien el algoritmo presenta un error matemático al manejar las variables de slack, al final de cuentas, estas son variables auxiliares que se emplean para determinar cuánto le falta a una solución (óptima o no) para llegar al tope permitido por la restricción respectiva de la cual salió la variable de slack. Es decir, para determinar el óptimo sólo necesitamos las variables de decisión ya que la función objetivo depende únicamente de estas.

Por lo tanto, a pesar de que ocurra ese error matemático al expresar el vector de respuesta, esto es irrelevante para lo cual nos importa dicha solución. El costo óptimo se puede calcular con las variables de decisión sin problema alguno. Es más, sin mucho esfuerzo sería posible reconstruir los valores de la SBF óptima mirando el tableau impreso en la última iteración.

En conclusión, es un algoritmo práctico y útil para lo que fue diseñado. Sí se recomienda su uso.

# Punto 2 - Problemas de Optimización

## Problema 1

Las variables de decisión del problema son: $x_{1} y x_{2}$ que corresponde al número de carros compactos y carros subcompactos, respectivamente.
Por otro lado, el problema de optimización original es:

$max (6,150x_{1}+5,500x_{2})$\
sujeto a \
$x_{1} \leq 1500$\
$x_{2} \leq 200$\
$250x_{1}+150x_{2} \leq 80,000$\
$18x_{1}+20x_{2} \leq 9,000$\
$x_{1},x_{2} \geq 0$

A continuación, se muestra la formulación del Problema de Programación Lineal en forma estándar

$mín (-6,150x_{1}-5,500x_{2})$\
sujeto a \
$x_{1}+y_{1} = 1500$\
$x_{2}+y_{2} = 200$\
$250x_{1}+150x_{2}+y_{3} = 80,000$\
$18x_{1}+20x_{2}+y_{4} = 9,000$\
$x_{1},x_{2},y_{1},y_{2},y_{3},y_{4} \geq 0$

In [ ]:
# Evaluar la solución del problema 1 empleando la función simplex
f = np.array([-6150, -5500, 0, 0, 0, 0])
A = np.array([[1, 0, 1, 0, 0, 0], [0, 1, 0, 1, 0, 0], [250, 150, 0, 0, 1, 0], [18, 20, 0, 0, 0, 1]])
b = np.array([1500, 200, 80000, 9000])

res_problema1 = simplex(f,A,b)

Starting Tableau:
ind 	x_0	x_1	s_0	s_1	s_2	s_3	
6	1.0	0.0	1.0	0.0	0.0	0.0	1500.0	
7	0.0	1.0	0.0	1.0	0.0	0.0	200.0	
8	250.0	150.0	0.0	0.0	1.0	0.0	80000.0	
9	18.0	20.0	0.0	0.0	0.0	1.0	9000.0	
	-6150.0	-5500.0	0.0	0.0	0.0	0.0	0.0	
--------------------------------------------------------------
ITERATION  1
ind 	x_0	x_1	s_0	s_1	s_2	s_3	
6	1.0	0.0	1.0	0.0	0.0	0.0	1500.0	
7	0.0	1.0	0.0	1.0	0.0	0.0	200.0	
8	250.0	150.0	0.0	0.0	1.0	0.0	80000.0	
9	18.0	20.0	0.0	0.0	0.0	1.0	9000.0	
	-6150.0	-5500.0	0.0	0.0	0.0	0.0	0.0	
Pivot Column: 1
Pivot Row: 2
Pivot Element:  250.0
--------------------------------------------------------------
ITERATION  2
ind 	x_0	x_1	s_0	s_1	s_2	s_3	
6	0.0	-0.6	1.0	0.0	-0.0	0.0	1180.0	
7	0.0	1.0	0.0	1.0	0.0	0.0	200.0	
0	1.0	0.6	0.0	0.0	0.0	0.0	320.0	
9	0.0	9.2	0.0	0.0	-0.07	1.0	3240.0	
	0.0	-1810.0	0.0	0.0	24.6	0.0	1968000.0	
Pivot Column: 2
Pivot Row: 1
Pivot Element:  1.0
--------------------------------------------------------------
ITERATION  3
ind 	x_0	x_1	s_0	s_1	s_2	

## Problema 2
Las varianles de decisión del problema son $x_{1},x_{2},x_{3},x_{4},x_{5}$, que corresponde a la cantidad de cada sabor de helado (Chocolate, Vainilla, Chicle, Banano y Lulo.)
Por otro lado, el problema de programación inicial es:

$max 1.3x_{1}+1.4x_{2}+1.3x_{3}+1.0x_{4}+1.2x_{5}$\
sujeto a\
$0.45x_{1}+0.5x_{2}+0.4x_{3}+0.4x_{4}+0.42x_{5} \leq 270$\
$0.5x_{1}+0.45x_{2}+0.4x_{3}+0.4x_{4}+0.2x_{5} \leq 197$\
$0.10x_{1}+0.15x_{2}+0.35x_{3}+0.2x_{4}+0.1x_{5} \leq 135$\
$x_{1},x_{2},x_{3},x_{4},x_{5},y_{1},y_{2},y_{3} \geq 0$

A continuación, se muestra la formulación del Problema de Programación Lineal en forma estándar

$mín -1.3x_{1}-1.4x_{2}-1.3x_{3}-1.0x_{4}-1.2x_{5}$\
sujeto a\
$0.45x_{1}+0.5x_{2}+0.4x_{3}+0.4x_{4}+0.42x_{5}+y_{1} = 270$\
$0.5x_{1}+0.45x_{2}+0.4x_{3}+0.4x_{4}+0.2x_{5}+y_{2} = 197$\
$0.10x_{1}+0.15x_{2}+0.35x_{3}+0.2x_{4}+0.1x_{5}+y_{3} = 135$\
$x_{1},x_{2},x_{3},x_{4},x_{5},y_{1},y_{2},y_{3} \geq 0$

In [ ]:
# Evaluar la solución del problema 1 empleando la función simplex
f = np.array([-1.3, -1.4, -1.3, -1.0, -1.2, 0, 0, 0])
A = np.array([[0.45, 0.5, 0.4, 0.4, 0.42, 1, 0, 0], [0.5, 0.45, 0.4, 0.4, 0.2, 0, 1, 0], [0.1, 0.15, 0.35, 0.2, 0.1, 0, 0, 1]])
b = np.array([270, 197, 135])

res_problema2 = simplex(f,A,b)

Starting Tableau:
ind 	x_0	x_1	x_2	x_3	x_4	s_0	s_1	s_2	
8	0.45	0.5	0.4	0.4	0.42	1.0	0.0	0.0	270.0	
9	0.5	0.45	0.4	0.4	0.2	0.0	1.0	0.0	197.0	
10	0.1	0.15	0.35	0.2	0.1	0.0	0.0	1.0	135.0	
	-1.3	-1.4	-1.3	-1.0	-1.2	0.0	0.0	0.0	0.0	
--------------------------------------------------------------
ITERATION  1
ind 	x_0	x_1	x_2	x_3	x_4	s_0	s_1	s_2	
8	0.45	0.5	0.4	0.4	0.42	1.0	0.0	0.0	270.0	
9	0.5	0.45	0.4	0.4	0.2	0.0	1.0	0.0	197.0	
10	0.1	0.15	0.35	0.2	0.1	0.0	0.0	1.0	135.0	
	-1.3	-1.4	-1.3	-1.0	-1.2	0.0	0.0	0.0	0.0	
Pivot Column: 2
Pivot Row: 1
Pivot Element:  0.45
--------------------------------------------------------------
ITERATION  2
ind 	x_0	x_1	x_2	x_3	x_4	s_0	s_1	s_2	
8	-0.11	0.0	-0.04	-0.04	0.2	1.0	-1.11	0.0	51.11	
1	1.11	1.0	0.89	0.89	0.44	0.0	2.22	0.0	437.78	
10	-0.07	0.0	0.22	0.07	0.03	0.0	-0.33	1.0	69.33	
	0.26	0.0	-0.06	0.24	-0.58	0.0	3.11	0.0	612.89	
Pivot Column: 5
Pivot Row: 0
Pivot Element:  0.19777777777777775
--------------------------------------------------------------
I